#Input and output changes for experiments

In [1]:
!pip install transformers -q
!pip install sentencepiece -q

     |████████████████████████████████| 3.1 MB 12.0 MB/s 
     |████████████████████████████████| 596 kB 39.9 MB/s 
     |████████████████████████████████| 895 kB 62.8 MB/s 
     |████████████████████████████████| 59 kB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 66.6 MB/s 
     |████████████████████████████████| 1.2 MB 14.2 MB/s 


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re

import sklearn


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Add column of all the text before the findings and impression sections

In [4]:
#load the file contain chexbert and chexpert annotations
tags_reports_df = pd.read_csv("/content/drive/My Drive/Final project - Zebra/Final data/chexbert&pert reports - train + test.csv")
tags_reports_df["start_report"] = np.nan
tags_reports_df.head()

,path,chexpert medical tags,report_impression,report_findings,start_report,chexbert medical tags
0,files/p10/p10000032/s50414267.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,"FINDINGS: There is no focal consolidation, ...",NaN,No Finding positive
1,files/p10/p10000032/s53189527.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary abnorm...,"FINDINGS: The cardiac, mediastinal and hila...",NaN,No Finding positive
2,files/p10/p10000032/s53911762.txt,No Finding positive,IMPRESSION: No acute intrathoracic process.,FINDINGS: Single frontal view of the chest ...,NaN,No Finding positive
3,files/p10/p10000032/s56699142.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,FINDINGS: The lungs are clear of focal cons...,NaN,No Finding positive
4,files/p10/p10000764/s57375967.txt,"Consolidation positive ,Pneumonia uncertain",IMPRESSION: Focal consolidation at the left...,FINDINGS: PA and lateral views of the chest...,NaN,"Consolidation positive ,Pneumonia uncertain"


In [ ]:
#findings section finder
findings_found = "FINDINGS"
impression_found = "IMPRESSION"
report_str = ""
flag = False

#iterate through all reports
for i in range (len(tags_reports_df)):
    path_report = tags_reports_df.loc[i,'path']
    print(path_report)
                
    report = open("/content/drive/My Drive/Final project - Zebra/Data/"+path_report, "r")
    
    #iterate through all lines in report
    for line in report:
      if findings_found in line:
        flag = True
      if impression_found in line:
        flag = True
      if flag == False:
        report_str += str(line)
      #if findings/impression word found then stop and append all words before that word
            
    
    #init all variables for the next report
    tags_reports_df.loc[i, 'start_report'] = report_str
    flag = False
    report_str = ""

#clean results   
tags_reports_df['start_report'] = tags_reports_df['start_report'].str.replace(r'\n', '')

tags_reports_df.head(15)

In [ ]:

from google.colab import drive
from google.colab import files
tags_reports_df.to_csv('chexbert&pert reports + all section before.csv', index = False)
files.download("chexbert&pert reports + all section before.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## delete parts/specific words from the reports that cause a lot of False positive like locations and history of the patients

In [ ]:
import re

#load the train file 
chexbert_df = pd.read_csv("/content/drive/My Drive/Final project - Zebra/Final data/chexbert&pert reports - train.csv")

#delete from the given report unwanted parts  
def patterns(fixed_sentence):
  start = re.escape("As")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("as")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("Compared")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("compared")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("In")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("since")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("most")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("through")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("Comparison")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("NOTIFICATION:")
  end   = re.escape("Dr. ___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("Dr.")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)


  start = re.escape(" on")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("from")
  end   = re.escape("___")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("findings")
  end   = re.escape("discovery")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  start = re.escape("Findings")
  end   = re.escape("discovery")
  pattern1 = '%s(.*?)%s' % (re.escape(start), re.escape(end))
  fixed_sentence = re.sub(pattern1,'', fixed_sentence)

  return fixed_sentence


def delete_parts(df, column_name):
  #delete parts/specific words from the given column that cause a lot of False positive like locations and history of the patients

  #Parameters:
  #    df :the given dataframe
  #    column_name :the given column

  #Returns:
  #    the column without the unwanted parts 

  fixed_study = ""
  words_to_remove = ['left ', 'right ', 'Left ', 'Right ' 'upper ', 'upper', 'Upper ', 'lower ', 'Lower ', 'right-sided ','left-sided '] #locations

  for study in df[column_name]:
    study = str(study)
    #print(study)
    for sentence in study.split("."):

      fixed_sentence = re.sub(r'|'.join(map(re.escape, words_to_remove)),'',sentence) # remove all locations
      if "NOTIFICATION:" in fixed_sentence:
        fixed_sentence = ""
      fixed_sentence = fixed_sentence+"." #add the deleted dots (removed before to split sentences)
      fixed_sentence = patterns(fixed_sentence) #delete all sub-sentences contain '___' and other
      fixed_sentence = re.sub(r' , ','',fixed_sentence) 
      fixed_sentence = re.sub(r'\(.\)','',fixed_sentence)
      fixed_study += fixed_sentence
      

    fixed_study = fixed_study[:-1] #remove the last extra dot in the end of the sentence

    df[column_name].iloc[count_study] = fixed_study
    fixed_study = ""

  return df

chexbert_df = delete_parts(chexbert_df, "report_impression")
chexbert_df = delete_parts(chexbert_df, "report_findings")
chexbert_df = delete_parts(chexbert_df, "start_report")



In [ ]:
from google.colab import drive
from google.colab import files
chexbert_df.to_csv('chexbert&pert reports - train - fixed labels.csv', index = False)
files.download("chexbert&pert reports - train - fixed labels.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>